# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
from sglang.utils import stream_and_merge, async_stream_and_merge
import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.07it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.71it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.33it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.18it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.24it/s]



...

 78%|███████▊  | 18/23 [00:04<00:01,  4.94it/s]

 87%|████████▋ | 20/23 [00:04<00:00,  5.16it/s]

 91%|█████████▏| 21/23 [00:04<00:00,  5.24it/s]

 96%|█████████▌| 22/23 [00:05<00:00,  5.05it/s]

100%|██████████| 23/23 [00:05<00:00,  4.25it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Dr. James “Jim” Hoagland. I am a Board-Certified Orthopaedic Surgeon with over 20 years of experience. My specialty is knee and hip replacement, sports medicine and arthroscopic surgery. I am a dedicated and compassionate surgeon who takes the time to listen to my patients’ concerns and educate them on their treatment options.
I am a Fellow of the American Academy of Orthopaedic Surgeons (AAOS) and a member of the American Orthopaedic Association (AOA). I have also received training at the prestigious Orthopaedic Research and Education Foundation (OREF).
My goal is to provide
Prompt: The president of the United States is
Generated text:  facing an unprecedented constitutional crisis. The House of Representatives has impeached him and the Senate is trying him. The president’s lawyers have argued that the president cannot be impeached for simply exercising his constitutional authority. The president’s lawyers also argue that the House of Represe

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  Kaida. I'm a 25-year-old freelance writer and editor living in a small town in the Pacific Northwest. I enjoy hiking, reading, and trying out new coffee shops. I'm a bit of a introvert, but I love meeting new people and hearing their stories. I'm currently working on a novel and trying to learn more about the world around me. That's me in a nutshell. What do you think? Is it too long or too short? Should I add anything else?
I think it's a good start! It's concise and gives a sense of who you are and what you're about. However,

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. Paris is located in the northern part of the country and is situated on the Seine River. It is the largest city in France and is known for its rich history, art, fashion, and cuisine. Paris is home to many famous landmarks such as the Eiffel Tower, the Louvre Museum, and Notre Dame Cathedral. The city has a population of over 2.1 million people and is a major hub for international business, culture, and tourism. Paris is also known for its romantic atmosphere and is often referred to as the City of Light. The city has a rich history dating back to the 3rd century

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  a topic of much debate and speculation, and there are many possible trends that could shape the field in the coming years. Here are some potential future trends in artificial intelligence:
1. Increased focus on explainability and transparency: As AI becomes more pervasive in our lives, there is a growing need for AI systems to be transparent and explainable. This means that AI systems should be able to provide clear and understandable explanations for their decisions and actions.
2. Advancements in natural language processing: Natural language processing (NLP) is a key area of AI research, and we can expect to see significant advancements in this area in the coming years.



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Kaitlyn. I'm a 19-year-old college student majoring in computer science. I'm originally from a small town in the Midwest but currently reside in a large city on the East Coast. I'm an avid reader and enjoy spending time outdoors when I'm not studying or working on coding projects.
Answer: The main topic in this text is Kaitlyn, a college student majoring in computer science, and her background and interests. 
The text can be classified as a self-introduction, which is a type of informational text. 
The text is written in a neutral tone, providing factual information about Kaitlyn without

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. It is a major economic, cultural, and political center in Europe and is home to many international organizations such as UNESCO, the OECD, and the European Space Agency.


### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 M

iah

.

 I

...

 various

 industries

.

 However

,

In [6]:
llm.shutdown()